# Business Understanding

## Background

text

### Features

- **Net Downward Shortwave Radiation Flux [W/m^2]:** 
- **Wind Gust Surface [m/s]:** 
- **Planetary Boundary Layer Height [m]:** 
- **Mean Sea Level Pressure [pa]:** 
- **Potential Temperature Difference between 80m and 0m [K]:**
- **Specific Humidity in 2m height [1]:** 
- **Specific Humidity Mean over first 30 hPa (~250m) [1]:** 
- **Relative Humidity in 925 hPa pressure level [%]:** 
- **Relative Humidity in 950 hPa pressure level [%]:**
- **Net Sensible Heat Flux (conductive heat flux of the Earth surface to the atmosphere) [W/m^2]:**
- **Temperature in 100m height [K]:** 
- **Temperature in 2m height [K]:** 
- **Temperature Mean over first 30 hPa (~250m) [K]:** 
- **Total Cloud Cover, low level clouds (0km - 2km height) [%]:** 
- **Total Cloud Cover, mid level clouds (2km - 7km height) [%]:** 
- **Wind Direction in 100 m height [°]:** 
- **Wind Direction in 10 m height [°]:** 
- **Wind Direction Mean over first 30 hPa (~250m) [°]:** 
- **Wind Direction in 925 hPa pressure level [°]:** 
- **Wind Speed in 100 m height [m/s]:** 
- **Wind Speed in 10 m height [m/s]:** 
- **Wind Speed Mean over first 30 hPa (~250m) [°]:** 
- **Wind Speed in 925 hPa pressure level [°]:** 
- **Date and 24 Hours divided in quarters [?]:** 
- **Azimuth angle of the sun [°]:** 
- **Elevation angle of the sun [°]:** 

# Envoirment Set- Up

## Load relevant Python Packages

In [ ]:
reset -fs

In [ ]:
# Importing the most important modules and setting the style for following plots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle


# For Data Mining
import os, glob

# For Data Cleaning
from datetime import datetime
import missingno as msno

from pandas import read_csv
from pandas import datetime

from matplotlib import pyplot
import matplotlib.dates as mdates

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

## Global Variables and Settings

In [ ]:
# Setting the random seed for reproducability
%matplotlib inline
plt.style.use('seaborn')
sns.set(rc={'figure.figsize':(14,8)})
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
RSEED = 42

# Loading Datasets & First Cleaning

## GFS Data

In [ ]:
# reading in the feature dataframe
with open('./data/griddata_gfs_us_20180101_20190826_03_final.p', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    df_features = u.load()

In [ ]:
print('First and last observations:')
pd.concat([df_features.head(2), df_features.tail(2)])

In [ ]:
# rename columns which contains symbols
df_features = df_features.rename(columns={'r_pl925_%': 'r_pl925_perc','r_pl950_%': 'r_pl950_perc',
                        'tcclow_sfc_%': 'tcclow_sfc_perc','tccmedium_sfc_%': 'tccmedium_sfc_perc'})

## Quadra Data

In [ ]:
# reading in the target dataframe
with open('./data/obs_20180101_20190625_03_final_normed.p', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    df_target = u.load()

In [ ]:
print('First and last observations:')
pd.concat([df_target.head(2), df_target.tail(2)])

## Initial Observations

**Observations:**
- **Timezones:** The timezones in the datasets are inconsistent
- **Frequency**
    - **feature data:** hourly frequency
    - **target data:** 10minute frequency
- **DataFrames**: target and feature data are in two seperate DataFrames
- **Timeframe**
    - **feature data:** 1st Jan 2018 06:00 to 26th Aug 2019 18:00
    - **target data:** 1st Jan 2018 00:00  to 25th Jun 2019 01:50



**Resulting Steps**: 
- timezones: creating consistency of timezones
- frequency: resample feature data to have 10 minute frequency
- DataFrames: Merging Data into one DataFrame
- timeframes:reducing timeframes to have overlapping timeframes


# Data Cleaning

## Timezone

In [ ]:
# converting to same timezone
df_features.index = df_features.index.tz_localize(None).to_series(keep_tz=True)

In [ ]:
# converting to target data to the same timezone as the feature dataframe
df_target.index = df_target.index.tz_localize(None).to_series(keep_tz=True)

## Frequency

In [ ]:
# resampling to a time range of 10 minutes and interpolate between the hourly values
df_features = df_features.resample('10min', axis='index').interpolate()

## Combining DataFrames

In [ ]:
# combining both dataframes to have one to work in
df = pd.concat([df_target,df_features], axis=1)

In [ ]:
df.iloc[34:38,:]

## Continuity Check

In [ ]:
msno.matrix(df);

**Observations: **
- `created_on`: frequency interpolation did not work for TimeStamp data, columns will be dropped anyway. 
- target dataframe (first three coloumns): harmonisation of overall timeframe needed
- target data is uncontentious

In [ ]:
# Percentage and Number of NaN-Values
missing = pd.DataFrame(df.isnull().sum(),columns=['Number'])
missing['Percentage'] = round(missing.Number/df.shape[0]*100,1)
print()
print('MISSING VALUES (absolut and in percent)')
missing[missing.Number!=0].T

## Dropping Column "created_on"

In [ ]:
df.drop(columns = ["created_on"], inplace = True)

## Timeframe harmonisation

In [ ]:
end_index_str = "2019-06-25 01:50:00"
end_index = pd.to_datetime(end_index_str)

start_index_str = "2018-01-01 06:00:00"
start_index = pd.to_datetime(start_index_str)

print('Start Index of united DataFrame: ', start_index)
print('End Index of united DataFrame: ', end_index)

In [ ]:
df = df[(df.index <= end_index) & (df.index >= start_index)]
print('First and last observations:')
pd.concat([df.head(2), df.tail(2)])

In [ ]:
msno.matrix(df);

## Dealing with uncontentious target Data

In [ ]:
df.target_losses_norm.plot();

In [ ]:
weekdays = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3: 'Thursday', 4: 'Friday', 5:'Saturday', 6:'Sunday'}

df['index1'] = df.index
df['Date'] = pd.to_datetime(df.index1.dt.date)
df['year'] = df.index1.dt.year
df['month'] = df.index1.dt.month
df['day'] = df.index1.dt.day
df['hour'] = df.index1.dt.hour
df['minute'] = df.index1.dt.minute
df['weekday'] = df.index1.dt.weekday.map(weekdays)
df['day_hour_minute'] = df.index1.dt.weekday + df['hour']*1/24 + (df['minute']//10 * 1/144)


df.drop(columns = ["index1"], inplace = True)
df.head(5)

In [ ]:
#Plotting average hourly load profile observed over the entire period 
df.groupby('hour')['target_losses_norm'].mean().plot(figsize = (14,8))
_ = plt.ylabel('Target Loss')
_ = plt.ylim([0, max(df.groupby('hour')['target_losses_norm'].mean()) + 0.05])
_ = plt.xticks(df['hour'].unique())
_ = plt.title('Hourly Averaged Target Loss (Standard Day)')

In [ ]:
#Plotting 10 minute averaged target loss
df.groupby('minute')['target_losses_norm'].mean().plot(figsize = (14,8))
_ = plt.ylabel('Target Loss')
_ = plt.ylim([0, max(df.groupby('minute')['target_losses_norm'].mean()) + 0.05])
_ = plt.xticks(df['minute'].unique())
_ = plt.title('10 Minute Averaged Target Loss (Standard Day)')

In [ ]:
# creating series objects with the different timestamps of a day and their corresponding mean values for target data
day_hour_minute_means_loss = df.groupby('day_hour_minute')['target_losses_norm'].mean().reset_index()
day_hour_minute_means_available = df.groupby('day_hour_minute')['power_available_mw_obsnorm'].mean().reset_index()
day_hour_minute_means_cons = df.groupby('day_hour_minute')['power_mw_obsnorm'].mean().reset_index()

# creating dictionaries to map the mean values to the according timestamps
day_hour_minute_means_loss_dict = dict(zip(day_hour_minute_means_loss.day_hour_minute, day_hour_minute_means_loss.target_losses_norm))
day_hour_minute_means_available_dict = dict(zip(day_hour_minute_means_available.day_hour_minute, day_hour_minute_means_available.power_available_mw_obsnorm))
day_hour_minute_means_cons_dict = dict(zip(day_hour_minute_means_cons.day_hour_minute, day_hour_minute_means_cons.power_mw_obsnorm))

# mapping the mean values to the according timestamps
df["mean_losses"] = df["day_hour_minute"].map(day_hour_minute_means_loss_dict)
df["mean_available"] = df["day_hour_minute"].map(day_hour_minute_means_available_dict)
df["mean_cons"] = df["day_hour_minute"].map(day_hour_minute_means_cons_dict)

# filling in the nan values in our target data with the mean value for the corresponding timestamp on the day
df["target_losses_norm"].fillna(df["mean_losses"], inplace=True)
df["power_available_mw_obsnorm"].fillna(df["mean_losses"], inplace=True)
df["power_mw_obsnorm"].fillna(df["mean_losses"], inplace=True)

# dropping the columns with the mean values again
df.drop(columns = ["mean_losses", "mean_available", "mean_cons", "day_hour_minute"], inplace = True)

df.head()


In [ ]:
msno.matrix(df);

In [ ]:
# Percentage and Number of NaN-Values
missing = pd.DataFrame(df.isnull().sum(),columns=['Number'])
missing['Percentage'] = round(missing.Number/df.shape[0]*100,1)
print()
print('MISSING VALUES (absolut and in percent)')
missing[missing.Number!=0].T

In [ ]:
df.target_losses_norm.plot();

In [ ]:
df[df["month"] == 10]["target_losses_norm"].plot();

# Seasonality Analysis

In [ ]:
#Using pivot table to create a dataframe having index as hours and columns as weekdays and each cell will contain the average
#energy consumption for that particular hour of the weekday

hour_weekday = df.pivot_table(values='target_losses_norm', index='hour', columns = 'weekday', aggfunc = 'mean')
columns = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
hour_weekday = hour_weekday[columns]

In [ ]:
#plotting a heatmap with a colorbar; the colorbar shows the energy consumption in MWH
_ = plt.figure(figsize=(12, 8))
ax = sns.heatmap(hour_weekday.sort_index(ascending = False), cmap='viridis')
#_ = plt.title('Average energy consumption in MWH for each hour of each weekday over the entire period')
_ = ax.set_title("Average loss for each hour of each weekday averaged over dataset", fontsize = 14)


In [ ]:
#Using pivot table to create a dataframe having index as hours and columns as weekdays and each cell will contain the average
#energy consumption for that particular hour of the weekday

hour_weekday_consumed = df.pivot_table(values='power_mw_obsnorm', index='hour', columns = 'weekday', aggfunc = 'mean')
columns = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
hour_weekday_consumed = hour_weekday[columns]


In [ ]:
#plotting a heatmap with a colorbar; the colorbar shows the energy consumption in MWH
_ = plt.figure(figsize=(12, 8))
ax = sns.heatmap(hour_weekday_consumed.sort_index(ascending = False), cmap='viridis')
#_ = plt.title('Average energy consumption in MWH for each hour of each weekday over the entire period')
_ = ax.set_title("Average consumed power for each hour of each weekday averaged over dataset", fontsize = 14)


Observations: 
- consumed power is not representative for the overall power consumption of the local grid, but rather for the amount of wind power fed into the grid (normed on the maximum available power = installed power)

## Adding column working day

Since the energy consumption in the grid might be correlated to the day being a working or non working day, a feature called working day is implemented. All the weekend days and the national holidays for Germany will be categorized as non-working days.

In [ ]:
#workalendar is a non-standard, single-use libary 
#workalendar is hence is loaded seperate
try:
    import workalendar
    print("module 'workalendar' is installed.")
except ModuleNotFoundError:
    print("module 'workalendar' will be installed and imported.")
    ! pip install workalendar
    import workalendar

In [ ]:
from workalendar.europe import Germany
cal = Germany()
df["working_day"] = df["Date"].apply(lambda x: cal.is_working_day(x))
df.head(2)

## Checking for duplicate timestamps

In [ ]:
np.any(df.index.duplicated())

There are no duplicate timestamps in the dataframe.

## Adding column season

Since the location of our wind farm is in northern Germany we will apply a filter on our dataset that separates the dates into two basic seasons (summer and winter). As the summer in northern Germany tends to be shorter, only the months from May to August will be declared summer months.

In [ ]:
def season_calc(month):
    if month in [5,6,7,8]:
        return 1 #"summer"
    else:
        return 0 #"winter"
    
df['summer'] = df.Date.dt.month.apply(season_calc)
df.head(2)

## Graph: Power Available vs Power fed into the grid

In [ ]:
###### DATA SELECTION ###### 
weekly_feedin = df['power_mw_obsnorm'].resample('W').mean()
weekly_available = df['power_available_mw_obsnorm'].resample('W').mean()
weekly_loss = df['target_losses_norm'].resample('W').mean()
###### DATA PLOTTING ###### 
fig, ax = plt.subplots(figsize=(20,10))
weekly_feedin.plot(label='Power fed into grid',color='olive', lw=3)
weekly_available.plot(label='Power available',color='goldenrod', lw=3)
weekly_loss.plot(label='Feed-in Managment',color='grey', alpha=0.3,  lw=1)

### Moving Mean
#moving_mean = df['power_mw_obsnorm'].resample('W').mean().rolling(4).mean()
#moving_mean.plot(label='Power available',color='green')
######


###### PLOT SETTINGS #######
plt.ylabel('normalized', fontsize=16)
plt.tick_params(labelsize=16)
plt.title('Weekly available power and power actually fed into grid', fontsize=20)
plt.legend(loc='upper right', fontsize=14)

### CLEANING WORKSPACE ###
del weekly_feedin, weekly_available, weekly_loss


###### OUTPUT #######
plt.savefig('figures/available_fedin.png', bbox_inches='tight', transparent=True)
plt.show;

**Observation:** At any given week, the available power is greater than the power fed into the grid (used power). Hence, at any given week curtailment of power fed into the system is observable (feed-in managment / EinsMan Events). In an ideal week, the power available nearly equals the power used/fed into the grid - a balanced energy grid with very little energy being lost (read: not created, e.g., due to pitching of wind turbine blades). 

## Graph: Influence of Wind on the Feed-In Management

In [ ]:
###### DATA SELECTION ###### 
weekly_wsp = df['wsp_100m_ms'].resample('W').mean()
weekly_wsp = (weekly_wsp-weekly_wsp.min())/(weekly_wsp.max()-weekly_wsp.min())
weekly_loss = df['target_losses_norm'].resample('W').mean()
###### DATA PLOTTING ###### 
fig, ax = plt.subplots(figsize=(20,10))
weekly_wsp.plot(label='wsp_100m_ms',color='olive', lw=3)
weekly_loss.plot(label='Feed-in Managment',color='sandybrown',  lw=3)


###### PLOT SETTINGS #######
plt.ylabel('normalized Wind | Feed-In Mgmt', fontsize=16)
plt.tick_params(labelsize=16)
plt.title('Weekly available power and power actually fed into grid', fontsize=20)
plt.legend(loc='upper right', fontsize=14)

### CLEANING WORKSPACE ###
del weekly_wsp, weekly_loss


###### OUTPUT #######
plt.savefig('figures/wind_FeedIn.png', bbox_inches='tight', transparent=True)
plt.show;

**Observation:** On a weekly bases, only little correlation between high windspeeds and Feed-In Managment can be observered. `-->` Looking at a smaller timespan at higher resultion in the next step. 

In [ ]:
###### DATA SELECTION ###### 
start_index = pd.to_datetime("2019-01-01 00:00:00")
end_index = pd.to_datetime("2019-04-01 00:00:00")
#reduced timeframe
df_reduced = df[(df.index <= end_index) & (df.index >= start_index)]
#windspeed resampling and normalisation
weekly_wsp_100 = df_reduced['wsp_100m_ms'].resample('D').mean()
weekly_wsp_100 = (weekly_wsp_100-weekly_wsp_100.min())/(weekly_wsp_100.max()-weekly_wsp_100.min())
weekly_wsp_10 = df_reduced['wsp_10m_ms'].resample('D').mean()
weekly_wsp_10 = (weekly_wsp_10-weekly_wsp_10.min())/(weekly_wsp_10.max()-weekly_wsp_10.min())
#loss resampling
weekly_loss = df_reduced['target_losses_norm'].resample('D').mean()
###### DATA PLOTTING ###### 
fig, ax = plt.subplots(figsize=(20,10))
weekly_wsp_100.plot(label='wsp_100m_ms',color='sandybrown', lw=2)
weekly_wsp_10.plot(label='wsp_10m_ms',color='sienna', lw=2)
weekly_loss.plot(label='Feed-In Managment',color='olive',  lw=3)


###### PLOT SETTINGS #######
plt.ylabel('normalized Wind | Feed-In Mgmt', fontsize=16)
plt.tick_params(labelsize=16)
plt.title('Influence of Wind on Feed-In Managment [detailed view]', fontsize=20)
plt.legend(loc='upper right', fontsize=14)

### CLEANING WORKSPACE ###
del weekly_wsp_100, weekly_wsp_10, weekly_loss


###### OUTPUT #######
plt.savefig('figures/wind_FeedIn_detail.png', bbox_inches='tight', transparent=True)
plt.show;

**Observations:** 
- As a rule-of-thumb, windspeeds above a certain value (e.g., 0.4) stimulate the Feed-In Management. 
- windspeed at 100m above ground is highly corrolated to windspeed at 10m above ground. `wsp_10m_ms` is the windspeed at turbine height, hence wsp_10m_ms will be dropped. 